In [40]:
import os
import json
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader

import utils
import config as cfg

from models import *

from torch.optim import SGD
import torch.nn as nn
import sklearn.metrics as metrics
from tqdm import tqdm

def get_files(PATH:str)->dict:
    file_dict = {'train':None, 'test':None}
    
    files = [os.path.join(PATH+'/train', file) \
        for file in os.listdir(PATH+'/train') if file.endswith('.json')]
    files.sort()
    file_dict['train'] = files
    
    files = [os.path.join(PATH+'/test', file) \
        for file in os.listdir(PATH+'/test') if file.endswith('.json')]
    files.sort()
    file_dict['test'] = files
    return file_dict

class DataManager():
    def __init__(self, files:list, is_train:bool=True, is_flat=True):
        self.files = files
        self.is_train = is_train
        self.global_train_data = {'x':[], 'y':[]}
        self.global_test_data = {'x':[], 'y':[]}
        
        for idx, file in enumerate(self.files):
            idx = str(idx)
            with open(file) as f:
                data = json.load(f)

                for user in data['users']:  
                    self.global_train_data['x'] = self.global_train_data['x'] + data['user_data'][user]['x']
                    self.global_train_data['y'] = self.global_train_data['y'] + data['user_data'][user]['y']
                    
                if not self.is_train:                   
                    for user in data['users']:
                        self.global_test_data['x'] = self.global_test_data['x'] + data['user_data'][user]['x']
                        self.global_test_data['y'] = self.global_test_data['y'] + data['user_data'][user]['y']

        if not is_flat:
            self.global_train_data['x'] = np.reshape(self.global_train_data['x'], (-1, 1, 28, 28))
            if not self.is_train:
                self.global_test_data['x'] = np.reshape(self.global_test_data['x'], (-1, 1, 28, 28))
                
class FEMNIST(Dataset):
    def __init__(self, data:dict):
        self.data = data
        self.data['x'] = np.array(data['x'])
        self.data['y'] = np.array(data['y'])
        
    def __getitem__(self, idx):
        self.X = torch.tensor(self.data['x'][idx,:]).float()
        self.Y = torch.tensor(self.data['y'][idx]).long()
        return self.X, self.Y

    def __len__(self):
        return len(self.data['y'])
    
    

In [41]:
class Args:
    def __init__(self):
        self.model = 'cnn'
        self.T_type = 0
        self.dataset = 'femnist'

In [42]:
args = Args()
args_dict  = utils.args2dict(args)
PATH       = cfg.DATAPATH['femnist']
    
file_dict  = get_files(PATH)
TRAIN_DM   = DataManager(file_dict['train'], is_train=True, is_flat=args.model=='dnn')
TEST_DM    = DataManager(file_dict['test'], is_train=False, is_flat=args.model=='dnn')

print("DATA READY")
    
print(f"WORKING WITH {cfg.DEVICE}")



DATA READY
WORKING WITH cuda


In [44]:
train_data = FEMNIST(TRAIN_DM.global_train_data)
test_data  = FEMNIST(TEST_DM.global_test_data)

In [45]:
train_loader = DataLoader(train_data, batch_size=256, shuffle=True, drop_last=True)
test_loader = DataLoader(test_data, batch_size=256, shuffle=False, drop_last=False)
    

In [53]:
T      = 0.1
EPOCHS = 200
LR     = 0.001
model = utils.get_model(args_dict['model'])
optimizer = SGD(params=model.parameters(), lr=LR)
criterion = nn.CrossEntropyLoss()

In [54]:
model.to(cfg.DEVICE)
for epoch in tqdm(range(EPOCHS)):
    train_loss_trace = []
    model.train()
    for idx, batch in enumerate(train_loader):
        optimizer.zero_grad()
        X, Y = batch
        X, Y = X.to(cfg.DEVICE), Y.to(cfg.DEVICE)
        pred = model(X) / T
        loss = criterion(pred, Y)
        loss.backward()
        train_loss_trace.append(loss.to('cpu').detach().numpy())
        optimizer.step()
    
    model.eval()
    with torch.no_grad():
        loss_trace, result_pred, result_anno = [], [], []
        for idx, batch in enumerate(test_loader):
            X, Y = batch
            X, Y = X.to(cfg.DEVICE), Y.to(cfg.DEVICE)
            pred = model(X) / T
            loss = criterion(pred, Y)
            loss_trace.append(loss.to('cpu').detach().numpy())
            pred_np  = pred.to('cpu').detach().numpy()
            pred_np  = np.argmax(pred_np, axis=1).squeeze()
            Y_np     = Y.to('cpu').detach().numpy().reshape(-1, 1).squeeze()
            result_pred = np.hstack((result_pred, pred_np))
            result_anno = np.hstack((result_anno, Y_np))
        
        acc = metrics.accuracy_score(y_true=result_anno, y_pred=result_pred)
        
    print(f'{epoch+1} epoch training loss = {np.average(train_loss_trace):.4f}')
    print(f'{epoch+1} epoch test loss     = {np.average(loss_trace):.4f}')
    print(f'{epoch+1} epoch test ACC      = {acc*100:.2f}')

  0%|          | 1/200 [00:05<17:18,  5.22s/it]

1 epoch training loss = 3.9768
1 epoch test loss     = 3.7243
1 epoch test ACC      = 5.62


  1%|          | 2/200 [00:10<17:30,  5.31s/it]

2 epoch training loss = 3.7176
2 epoch test loss     = 3.6557
2 epoch test ACC      = 14.15


  2%|▏         | 3/200 [00:15<17:19,  5.28s/it]

...

198 epoch training loss = 0.6418
198 epoch test loss     = 0.5476
198 epoch test ACC      = 82.56


100%|█████████▉| 199/200 [16:55<00:05,  5.13s/it]

199 epoch training loss = 0.6448
199 epoch test loss     = 0.5484
199 epoch test ACC      = 82.31


100%|██████████| 200/200 [17:00<00:00,  5.10s/it]

200 epoch training loss = 0.6415
200 epoch test loss     = 0.5475
200 epoch test ACC      = 82.49


T=1 200 epoch -> 74.63